In [1]:
# 라이브러리 선언
import requests
import bs4
import pandas as pd
import myFunction as mf
from lxml import html
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus, unquote

In [2]:
baseUrl = "https://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getVilageFcst"
inApiKey = "txTdKQwpVPsjw4+Ft28LtAJwIUQVgGgpTkA8yS30WrDnbOuAn6M46TJUySvewlPJhxqQptIEPLONoz+EZuK0lw=="
inBaseDate = mf.todayInfo()
inBaseTime = mf.timeInfo()
inNumRows = "12"
inPageNum = "1"
inNx = "55"
inNy = "127"

queryParams = '?' + urlencode(
    {
        quote_plus('ServiceKey') : inApiKey,
        quote_plus('numOfRows') : inNumRows,
        quote_plus('pageNo') : inPageNum,
        quote_plus('base_date') : inBaseDate,
        quote_plus('base_time') : inBaseTime,
        quote_plus('nx') : inNx,
        quote_plus('ny') : inNy
     }
)

# 위 변수들을 조합하여 최종적으로 접근할 Url 산출
targetUrl = baseUrl + queryParams

# 확인용 print문
# print(targetUrl)

# 현재 해당 페이지에 접근할 때 SSL 인증 실패 발생. verify 를 false 로 주어서 경고 무시
try:
    resp = requests.get(targetUrl, verify = False)
except Exception as e:
    print(e)

# resp를 xml 파싱
bs = bs4.BeautifulSoup(resp.text, "lxml-xml")

# 파싱한 결과물을 items 태그로 좁힘
itemsTag = bs.find("items")

# items 태그로 좁힌 것들 중 item 태그를 가진 것들을 전부 list 형태로 추출
itemTag = itemsTag.findAll("item")

C:\Users\myWin\anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'apis.data.go.kr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [3]:
# 뒤에서 최종결과로 나온 리스트에 추가할 칼럼값 2개 추출
fcstTime = itemTag[0].fcstTime.text
fcstDate = itemTag[0].fcstDate.text

# 카테고리 값과 예보 값을 담을 리스트 선언
categoryList = []
fcstValueList = []

# 각각의 내용물만 받을 수 있도록 반복문 실행
for i in range(0, len(itemTag)):
    categoryList.append(itemTag[i].category.text)
    fcstValueList.append(itemTag[i].fcstValue.text)

# 반복문 결과 확인용 print문
# print(categoryList)
# print(fcstValueList)

# convertColName 함수를 이용하여 category명을 전부 의미하는 내용으로 변환
nameList = mf.convertColName(categoryList)

# 확인용 print문
# print(nameList)

# 칼럼명 2개를 추가해주면 되는 것들이기에 insert로 별도 추가
nameList.insert(0, "예보일자")
nameList.insert(1, "예보시각")

# 각 데이터의 가장 앞에 붙여주면 되는 것들이기에 insert로 별도 추가
fcstValueList.insert(0, fcstDate)
fcstValueList.insert(1, fcstTime)

# 예보의 값 확인용 print문
# print(fcstValueList)

# 하나의 행으로 만들기 위하여 새로 리스트 선언 후, 값들이 들은 fcstValueList를 append
rawList = []

rawList.append(fcstValueList)

In [5]:
# nameList를 column으로 주고 DataFrame으로 출력
pd.DataFrame(rawList, columns = nameList)

,예보일자,예보시각,1시간 기온 (단위 : ℃),풍속/동서성분 (단위 : m/s),풍속/남북성분 (단위 : m/s),풍향 (단위 : deg),풍속 (단위 : m/s),하늘상태,강수형태,강수확률 (단위 : %),파고 (단위 : M),시간 당 강수량 (단위 : mm),습도 (단위 : %),시간 당 적설량 (단위 : cm)
0,20220419,0000,10,1.9,0.1,267,1.9,3,0,20,0,강수없음,80,적설없음
